In [1]:
import rospy
from sensor_msgs.msg import Image
import ros_numpy
import numpy as np
import cv2
from threading import Thread

rospy.init_node('publish', anonymous=True)

In [2]:
publishers = [
	rospy.Publisher(url, Image, queue_size=1) for url in [
		'/camera_101/image_raw',
		'/camera_102/image_raw',
	]
]

In [3]:
class Camera(Thread):
	def __init__(self, url: str, publisher: rospy.Publisher) -> None:
		super().__init__()

		self.url = url
		self.publisher = publisher
		self.running = False
		self.cap = cv2.VideoCapture(self.url)
		self.rate = rospy.Rate(1)
 
		# Check if camera opened successfully
		if (self.cap.isOpened()== False): 
			print("Error opening video stream or file")
	
	def run(self):
		self.running = True
		while(self.running and self.cap.isOpened()):
			# Capture frame-by-frame
			ret, frame = self.cap.read()
			if ret == True:
				image = np.array(frame)
				# image = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (384, 192), cv2.INTER_LINEAR)
				image = ros_numpy.msgify(Image, image, encoding='bgr8')
				self.publisher.publish(image)
				self.rate.sleep()
		
			# Break the loop
			else: 
				break
		
		# When everything done, release the video capture object
		self.cap.release()
	
	def stop(self):
		self.running = False

In [4]:
cameras = [Camera(url, publisher) for (publisher, url) in zip(publishers, [
	'rtsp://192.168.31.233:8554/cam1',
	'rtsp://192.168.31.233:8554/cam2'
])]

In [5]:
for camera in cameras:
	camera.start()

In [6]:
for camera in cameras:
	camera.stop()

--------

In [1]:
import rospy
from sensor_msgs.msg import Image
import ros_numpy
import numpy as np
import cv2
from PIL import Image as PILImage

rospy.init_node('publish', anonymous=True)

[WARN] [1722580511.479156]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
[WARN] [1722581930.064456]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.


In [2]:
publish = rospy.Publisher('/camera_101/image_raw', Image, queue_size=1)
publish_2 = rospy.Publisher('/camera_102/image_raw', Image, queue_size=1)

In [15]:
W, H = 960, 480

image = PILImage.open('cityscapes_demo.png')
image = np.array(image)
image = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (W, H), cv2.INTER_LINEAR)
image_msg = ros_numpy.msgify(Image, image, encoding='bgr8')

In [16]:
publish.publish(image_msg)
publish_2.publish(image_msg)